In [ ]:
import time
import pandas as pd
import os
import numpy as np
import snapatac2_scooby as sp
import scanpy as sc
import anndata as ad
import glob

import scipy.sparse
import tqdm

Important:
* SnapATAC automatically deletes reads that have the same start end and UMI (duplicates)
* If you do not provide UMI identifier (umi_regex or umi_tag) it will delete all reads that have the same start/end (not ideal)

## Read in RNA bam files

In [2]:
data_path = 'tmp'

In [4]:
adata = sc.read(
    os.path.join(data_path, 'bmmc_multiome_multivi_neurips21_curated.h5ad'), backed='r')

/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [6]:
adata.obs['sample'] = adata.obs['old_neurips21_obs_names'].str.split('-').str[-1]
adata.obs['barcode'] = adata.obs['old_neurips21_obs_names'].str.split('-').str[0] + '-1'

### Make Fragment file

In [33]:
sample_files = glob.glob(os.path.join(data_path , 'filtered_bam/*bam'))

In [40]:
for sample_file in tqdm.tqdm(sample_files):
    print(sample_file)
    sample = os.path.basename(sample_file).split('_tag.bam')[0]
    print(sample)
    out_path = os.path.join(data_path, 'snapatac', 'fragments')
    outfile = os.path.join(out_path, f'{sample}.fragments.bed.gz')
    whitelist = adata.obs.query("sample == @sample")['barcode'].to_list()
    sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]
    
    print(sample_number)
    if (len(whitelist)>0) & (~os.path.exists(os.path.join(out_path, f'{sample}.fragments.bed.minus.gz'))):
        sp.pp.make_fragment_file(
            sample_file, 
            output_file=outfile,
            barcode_tag="CB", 
            umi_tag="UB",
            umi_regex=None, 
            stranded=True, 
            is_paired=False, 
            shift_left=0, 
            shift_right=0
        )
        

    for strand in ['plus', 'minus']:
        if not os.path.exists(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_{sample}_{strand}.h5ad')):
            test = sp.pp.import_data(
                    f"{out_path}/{sample}.fragments.bed.{strand}.gz", 
                    chrom_sizes=sp.genome.hg38, 
                    min_num_fragments=0, 
                    n_jobs=-1,
                    whitelist=whitelist
                )
            if not test.obs_names.isin(whitelist).all():
                print(sample)
                break
            if not sample == 's2d4':
                test.obs.index = test.obs.index.str.split('-1').str[0] + '-' + sample_number + '-' + sample
            else:
                test.obs.index = test.obs.index.str.split('-1').str[0] + '-' + sample
                
            test.X = scipy.sparse.csr_matrix((test.obsm['fragment_single'].shape[0], 0))
            test.write(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_{sample}_{strand}.h5ad'))
    else:
        print(f"Skipping {sample}")

  0%|          | 0/13 [00:00<?, ?it/s]/scratch/tmp/martensl/ipykernel_4028376/2272645163.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]


/s/project/QNA/scborzoi/neurips_bone_marrow/filtered_bam/s2d4_tag.bam
s2d4
s2d4


  8%|▊         | 1/13 [00:25<05:05, 25.42s/it]

Skipping s2d4
/s/project/QNA/scborzoi/neurips_bone_marrow/filtered_bam/s4d8_tag.bam
s4d8
14


/scratch/tmp/martensl/ipykernel_4028376/2272645163.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]
 15%|█▌        | 2/13 [00:37<03:16, 17.85s/it]

...

Skipping s3d6
/s/project/QNA/scborzoi/neurips_bone_marrow/filtered_bam/s2d5_tag.bam
s2d5
6


/scratch/tmp/martensl/ipykernel_4028376/2272645163.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]
 92%|█████████▏| 12/13 [02:44<00:12, 12.49s/it]

Skipping s2d5
/s/project/QNA/scborzoi/neurips_bone_marrow/filtered_bam/s1d1_tag.bam
s1d1
1


/scratch/tmp/martensl/ipykernel_4028376/2272645163.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]
100%|██████████| 13/13 [03:00<00:00, 13.87s/it]

Skipping s1d1


## Combine both samples

In [42]:
samples = [os.path.basename(sample_file).split('_tag.bam')[0] for sample_file in sample_files]

In [53]:
for strand in ['plus', 'minus']:
    print(strand)
    adatas = [
        os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_{sample}_{strand}.h5ad') 
     for sample in samples
    ]

    anndata.experimental.concat_on_disk(
        in_files=adatas, 
        out_file=os.path.join(data_path, 'snapatac','anndata', f'snapatac_merged_{strand}.h5ad'),
        uns_merge='unique'
    )

    # uns merge is not working, add that

    adata_cov = sc.read_h5ad(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_{strand}.h5ad'))

    test = sc.read(adatas[0])

    adata_cov.uns = test.uns.copy()

    # join obs
    adata_cov.obs = adata_cov.obs.join(adata.obs.set_index('old_neurips21_obs_names'))
    
    #reorder
    adata_cov = adata_cov[adata.obs['old_neurips21_obs_names']]
    adata_cov.write(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_{strand}.h5ad'))

plus


/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-pack

minus


/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-pack

## Read in ATAC fragment files

In [73]:
sample_files = glob.glob(os.path.join(data_path , '*', '*atac_fragments.tsv.gz'))

In [75]:
for sample_file in tqdm.tqdm(sample_files):
    print(sample_file)
    sample = sample_file.split('/')[-2]
    print(sample)
    whitelist = adata.obs.query("sample == @sample")['barcode'].to_list()
    sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]


    if not os.path.exists(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_{sample}_atac.h5ad')):
        test = sp.pp.import_data(
                sample_file, 
                chrom_sizes=sp.genome.hg38, 
                min_num_fragments=0, 
                n_jobs=-1,
                whitelist=whitelist,
                sorted_by_barcode=False
            )
        if not test.obs_names.isin(whitelist).all():
            print(sample)
            break
        if not sample == 's2d4':
            test.obs.index = test.obs.index.str.split('-1').str[0] + '-' + sample_number + '-' + sample
        else:
            test.obs.index = test.obs.index.str.split('-1').str[0] + '-' + sample

        test.X = scipy.sparse.csr_matrix((test.obsm['fragment_paired'].shape[0], 0))
        test.write(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_{sample}_atac.h5ad'))
    else:
        print(f"Skipping {sample}")

  0%|          | 0/13 [00:00<?, ?it/s]/scratch/tmp/martensl/ipykernel_4028376/1774371288.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]


/s/project/QNA/scborzoi/neurips_bone_marrow/s3d7/atac_fragments.tsv.gz
s3d7
Skipping s3d7
/s/project/QNA/scborzoi/neurips_bone_marrow/s1d2/atac_fragments.tsv.gz
s1d2


 15%|█▌        | 2/13 [02:27<13:32, 73.87s/it]/scratch/tmp/martensl/ipykernel_4028376/1774371288.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]


/s/project/QNA/scborzoi/neurips_bone_marrow/s4d1/atac_fragments.tsv.gz
s4d1


 23%|██▎       | 3/13 [06:32<24:09, 144.90s/it]/scratch/tmp/martensl/ipykernel_4028376/1774371288.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]


...

 85%|████████▍ | 11/13 [34:39<07:21, 220.56s/it]/scratch/tmp/martensl/ipykernel_4028376/1774371288.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]


/s/project/QNA/scborzoi/neurips_bone_marrow/s3d3/atac_fragments.tsv.gz
s3d3


 92%|█████████▏| 12/13 [42:52<05:03, 303.23s/it]/scratch/tmp/martensl/ipykernel_4028376/1774371288.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample_number = adata.obs.query("sample == @sample")['old_neurips21_obs_names'][0].split('-')[1]


/s/project/QNA/scborzoi/neurips_bone_marrow/s1d3/atac_fragments.tsv.gz
s1d3


100%|██████████| 13/13 [44:48<00:00, 206.77s/it]


In [77]:
    
adatas = [
    os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_{sample}_atac.h5ad') 
 for sample in samples
]

anndata.experimental.concat_on_disk(
    in_files=adatas, 
    out_file=os.path.join(data_path, 'snapatac','anndata', f'snapatac_merged_atac.h5ad'),
    uns_merge='unique'
)

# uns merge is not working, add that

adata_cov = sc.read_h5ad(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_atac.h5ad'))

test = sc.read(adatas[0])

adata_cov.uns = test.uns.copy()

# join obs
adata_cov.obs = adata_cov.obs.join(adata.obs.set_index('old_neurips21_obs_names'))

#reorder
adata_cov = adata_cov[adata.obs['old_neurips21_obs_names']]
adata_cov.write(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_atac.h5ad'))

/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/anndata.py:1209: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-pack

## Save insertions

In [ ]:
adata_cov = sc.read(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_atac_fragments.h5ad'))

/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [82]:
adata_cov 

AnnData object with n_obs × n_vars = 69247 × 0
    obs: 'n_fragment', 'frac_dup', 'frac_mito', 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'GEX_pseudotime_order', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_pseudotime_order', 'DonorID', 'VendorLot', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'modality', 'site', 'donor', 'batch', 'l1_cell_type', 'l2_cell_type', 'neurips21_cell_type', 'frag_file_bcs', 'sample', 'barcode'
    uns: 'reference_sequences'
    obsm: 'fragment_paired'

In [83]:
# From scprinter!
from scipy.sparse import csr_matrix
import numpy as np
def frags_to_insertions(data, split=False):
    x = data.obsm["fragment_paired"]
    insertion = csr_matrix(
        (
            np.ones(len(x.indices) * 2, dtype="uint16"),
            np.stack([x.indices, x.indices + x.data], axis=-1).reshape((-1)),
            x.indptr * 2,
        ),
        shape=x.shape,
    )
    insertion.sort_indices()
    insertion.sum_duplicates()
    if split:
        indx = list(
            np.cumsum(data.uns["reference_sequences"]["reference_seq_length"]).astype("int")
        )
        start = [0] + indx
        end = indx
        for chrom, start, end in zip(
            data.uns["reference_sequences"]["reference_seq_name"], start, end
        ):
            data.obsm["insertion_%s" % chrom] = insertion[:, start:end].tocsc()

    else:
        data.obsm["insertion"] = insertion
    # data.obsm['insertion'] = insertion
    return data

In [84]:
adata_cov = frags_to_insertions(adata_cov)

In [85]:
adata_cov.X = scipy.sparse.csr_matrix((adata_cov.obsm['insertion'].shape[0], 0))

In [86]:
adata_cov.obsm.pop('fragment_paired')

<69247x3088269832 sparse matrix of type '<class 'numpy.uint32'>'
	with 887087404 stored elements in Compressed Sparse Row format>

In [87]:
adata_cov.write(os.path.join(data_path, 'snapatac', 'anndata', 'snapatac_merged_atac.h5ad'))

## Filter out doublets and other myeloids

In [24]:
import scanpy as sc
import os
import numpy as np
import snapatac2 as sp

In [7]:
data_path = '/s/project/QNA/scborzoi/neurips_bone_marrow'

In [8]:
adata = sc.read(
    os.path.join(data_path, 'bmmc_multiome_multivi_neurips21_curated_new_palantir_fixed.h5ad'), 
    backed='r')

/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [13]:
for name in ['plus', 'minus', 'atac']:
    print(name)
    snap = sc.read(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_{name}.h5ad'))
    snap = snap[adata.obs_names]
    print(snap.shape)
    snap.write(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_fixed_{name}.h5ad'))

plus


/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


(63683, 0)
minus


/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


(63683, 0)
atac


/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


(63683, 0)


In [9]:
adata_minus = sc.read(os.path.join(data_path, 'snapatac', 'anndata', 'snapatac_merged_minus.h5ad'))

/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


### Write out pseudobulk coverage atac

In [4]:
adata_cov = sc.read(os.path.join(data_path, 'snapatac', 'anndata', 'snapatac_merged_fixed_atac.h5ad'))

/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/opt/modules/i12g/anaconda/envs/scprinter_env/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [6]:
adata_cov.obsm['fragment_single'] = adata_cov.obsm['insertion']

In [7]:
adata_cov.obsm['fragment_single'].data = np.ones(adata_cov.obsm['fragment_single'].data.shape, dtype=np.int32)

In [ ]:
sp.ex.export_coverage(
            adata_cov, 
            groupby='l2_cell_type', 
            bin_size=1, 
            out_dir=os.path.join(data_path, 'snapatac',f"pseudobulks_fixed"), 
            normalization=None,
            n_jobs=-1,
            max_frag_length=None,
            suffix=f'.bw',
            prefix=f"atac."
        )

2024-07-12 18:19:17 - INFO - Exporting fragments...


### Write pseudobulks RNA

In [ ]:
for strand in ['plus', 'minus']:
    adata_cov = sc.read(os.path.join(data_path, 'snapatac', 'anndata', f'snapatac_merged_fixed_{strand}.h5ad'))
    sp.ex.export_coverage(
        adata_cov, 
        groupby='l2_cell_type', 
        bin_size=1, 
        out_dir=os.path.join(data_path, 'snapatac',"pseudobulks_fixed"), 
        normalization=None,
        n_jobs=-1,
        max_frag_length=None,
        suffix='.bw',
        prefix=f"{strand}."
    )